### Importação do pacote para uso de cubos do Microstrategy

In [1]:
from mstrio import microstrategy
import pickle

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import numpy

In [ ]:
conn = microstrategy.Connection(base_url="http://aesc-biweb-01.hmd.local:8080/MicroStrategyLibrary/api", username="administrator", password="ferias2018", project_name="IA")
conn.connect()

### Leitura dos Datasets

In [ ]:
agenda2 = conn.get_report(report_id='D4FADEF74C36C94C470776A7B581B030')
produtos2 = conn.get_report(report_id='F2C5D89B4A8DB6AAD16F1A9EC9230829')
procedimentos2 = conn.get_report(report_id='4D0CC4FB4C5B18466D64FBA072B1AD27')

### Preciso exportar para conseguir que os objetos ficam com o mesmo datatype

In [ ]:
agenda2.to_excel('obs_2019.xlsx')
agenda2.to_excel('prod_2019.xlsx')
agenda2.to_excel('proc_2019.xlsx')

### Leitura do excel recem exportado.

In [ ]:
cut_num = 30000
agenda = pd.read_excel('obs_2019.xlsx', nrows=cut_num)
produtos = pd.read_excel('prod_2019.xlsx') 
procedimentos = pd.read_excel('proc_2019.xlsx')

In [ ]:
agenda = agenda.astype({'CD_AVISO_CIRURGIA':int})
#agenda2 = agenda2.astype({'CD_AVISO_CIRURGIA':int})

### Junção dos datasets.;

In [ ]:
import sys
for idx in agenda.index:
    cd_aviso = agenda.loc[idx, 'CD_AVISO_CIRURGIA']
    prods = produtos[produtos['CD_AVISO_CIRURGIA'] == cd_aviso]
    procs = procedimentos[procedimentos['CD_AVISO_CIRURGIA'] == cd_aviso]
    
    if pd.isnull(agenda.loc[idx, 'DS_OBSERVACAO']):
        agenda.loc[idx, 'DS_OBSERVACAO'] = ''
    if len(prods) > 0:
        agenda.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(prods['DS_PRODUTO'].astype('str').values)
    if len(procs) > 0:
        agenda.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(procs['PROCEDIMENTO'].astype('str').values)
    if (idx % 100) == 0:
        sys.stdout.write(str(idx) + ', ')
        sys.stdout.flush()
    if (idx % 2000) == 0:
        print('')

  

### Configuração do dataset

In [ ]:
import numpy as np
num_features = 500
#vectorizer = TfidfVectorizer(max_features=num_features).fit(agenda['DS_OBSERVACAO'].values)
vectorizer = pickle.load(open('agenda_vectorizer.pckls', 'rb'))
data = vectorizer.transform(agenda['DS_OBSERVACAO'].values)
data = data.todense()
## adiciona idade
mean_idade = int(np.mean(agenda['VL_IDADE']))
values = {'VL_IDADE': mean_idade}
agenda.fillna(value=values, inplace=True)
idades = agenda['VL_IDADE'].astype('int').values.reshape(data.shape[0],1)
data = np.append(data, idades, axis=1)
data.shape


### Criação dos dados de treino e teste e aplicação de RandomForest

In [ ]:
classifier = pickle.load(open('agenda_classifier.pckls', 'rb'))
#scores = cross_val_score(RandomForestClassifier(n_estimators=50), data, agenda['SN_UTI'])
#classifier = RandomForestClassifier(n_estimators=50).fit(data, agenda['SN_UTI'].values)
#scores, np.mean(scores)



In [ ]:
features = {}

### Determina o peso das features

In [ ]:
import operator
features = {}
ridx = {idx:term for term,idx in vectorizer.vocabulary_.items()}
for i,v in enumerate(classifier.feature_importances_):
    if i >= num_features:
        features['IDADE'] = v
        break
        
    if not ridx[i] in features.keys():
        features[ridx[i]] = v
    else:
        features[ridx[i]] += v
        if (i % 100) == 0:
            sys.stdout.write(str(i) + ', ')
            sys.stdout.flush()


In [ ]:
sorted(features.items(), key=operator.itemgetter(1), reverse=True)[:20]

### Exporta o modelo

In [ ]:
import pickle
#pickle.dump(classifier, open('agenda_classifier.pckls', 'wb'))
#pickle.dump(vectorizer, open('agenda_vectorizer.pckls', 'wb'))
#agenda.to_csv('agenda.csv.gz', compression='gzip', index=None)
#from google.colab import files
#files.download('agenda.csv.gz')

### Aplica o modelo

In [ ]:
agendaTeste = pd.read_excel('obs_2019.xlsx')

agendaTeste.columns = agenda.columns

for idx in agendaTeste.index:
    cd_aviso = agendaTeste.loc[idx, 'CD_AVISO_CIRURGIA']
    prods = produtos[produtos['CD_AVISO_CIRURGIA'] == cd_aviso]
    procs = procedimentos[procedimentos['CD_AVISO_CIRURGIA'] == cd_aviso]
   
    #if len(prods) > 0:

        #agendaTeste.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(prods['DS_PRODUTO'].astype)

    #if len(procs) > 0:
        #agendaTeste.loc[idx, 'DS_OBSERVACAO'] += ' ' + (' ').join(procs['PROCEDIMENTO'].astype)
        
    if (idx % 100) == 0:
        sys.stdout.write(str(idx) + ', ')
        sys.stdout.flush()


In [ ]:
data_teste = vectorizer.transform(agendaTeste['DS_OBSERVACAO'].astype('str').values)
data_teste = data_teste.todense()

In [ ]:
mean_idade = int(np.mean(agendaTeste['VL_IDADE']))
values = {'VL_IDADE': mean_idade}
agendaTeste.fillna(value=values, inplace=True)
idades = agendaTeste['VL_IDADE'].astype('int').values.reshape(data_teste.shape[0],1)

data_teste = np.append(data_teste, idades, axis=1)
data_teste.shape

In [ ]:
agendaTeste['pred'] = classifier.predict(data_teste)

proba_ar = classifier.predict_proba(data_teste)
agendaTeste['pred_0'] = proba_ar[:,0]
agendaTeste['pred_1'] = proba_ar[:,1]

### Resultados

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(agendaTeste['SN_UTI'], agendaTeste['pred'].values))

In [ ]:
agenda2['DATA_CIR2'] = pd.to_datetime(agendaTeste['DATA_CIR']).dt.strftime('%d/%m/%Y')

### Publica Resultados

In [ ]:
novo_cubo=0

if novo_cubo == 1:
    newDatasetIdmes, newTableIdmes = conn.create_dataset(data_frame= agendaTeste, dataset_name='Cubo_Result_pred_CTI', table_name='Agenda')
else:
    conn.update_dataset(data_frame=agendaTeste, dataset_id='E7FD43FC473B345D98C33DBB15158C63', table_name='Agenda', update_policy='replace')




In [ ]:
print("a")